In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train import *
from eval import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)


# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



/Users/nikolaushouben/opt/anaconda3/envs/Wattcast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [2]:
def train_eval_light():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model_instance = get_model(config)
    model_instance, _ = train_models([model_instance], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model_instance[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



In [3]:
# See what keys are in the h5py data file
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

print("Locations: ", locations_dict)
print("Resolutions: ", resolutions_dict)

for scale, locations in locations_dict.items():
    if scale == '4_neighborhood.h5':
        for location in locations:
            df_train = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/train_target')
            df_val = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/val_target')
            df_test = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/test_target')
            fig = px.line(df_train, title=f'{scale}: {location}')
            fig.add_trace(px.line(df_val, title='Validation Set').data[0])
            fig.add_trace(px.line(df_test, title='Test Set').data[0])
            fig.show()



Locations:  {'5_building.h5': ['building_0', 'building_1', 'building_2'], '4_neighborhood.h5': ['neighborhood_0', 'neighborhood_1', 'neighborhood_2'], '3_village.h5': ['village_0', 'village_1', 'village_2'], '2_town.h5': ['town_0', 'town_1', 'town_2'], '1_county.h5': ['Los_Angeles', 'New_York', 'Sacramento']}
Resolutions:  {'5_building.h5': ['60min'], '4_neighborhood.h5': ['60min'], '3_village.h5': ['15min', '60min'], '2_town.h5': ['15min', '60min'], '1_county.h5': ['60min']}


### Running

In [ ]:
# run parameters

sweeps = 20

scale_location_pairs = (
    # ('1_county', 'Sacramento'),
    # ('1_county', 'New_York'),
    #('2_town', 'town_0'),
    # ('2_town', 'town_1'),
    # ('2_town', 'town_2'),
    # ('3_village', 'village_1'),
    # ('3_village', 'village_2'),
    #('3_village', 'village_0'),
    # ('4_neighborhood', 'neighborhood_0'),
    # ('4_neighborhood', 'neighborhood_1'),
    # ('4_neighborhood', 'neighborhood_2'),
    # ('5_building', 'building_0'),
    # ('5_building', 'building_1'),
    ('5_building', 'building_2'),
      )



models = [
        #'rf',
         'xgb', 
        # 'gru', 
        #'lgbm',  
        # 'nbeats',
        #'tft'
        ]


for scale, location in scale_location_pairs:

    for model in models:
        # placeholder initialization of config file (will be updated in train_eval_light())
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': True,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
        }
        
        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="WattCast_tuning")
        wandb.agent(sweep_id, train_eval_light, count=sweeps)


### Degbugging

In [4]:
model = 'rf'
scale = '4_neighborhood'
location = 'neighborhood_1'


config_run = {
    'spatial_scale': scale,
    'temp_resolution': 60,
    'location': location,
    'model': model,
    'horizon_in_hours': 24,
    'lookback_in_hours': 24,
    'boxcox': True,
    'liklihood': None,
    'weather': True,
    'holiday': True,
    'datetime_encodings': False,
}

with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
    sweep_config = json.load(fp)   


config = Config().from_dict(config_run)


In [5]:
pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

In [8]:
model_instance = get_model(config)

In [9]:
type(model_instance)

darts.models.forecasting.random_forest.RandomForest

In [10]:
model_instance, _ = train_models([model_instance], ts_train_piped, ts_train_weather_piped, 
                                    ts_val_piped, ts_val_weather_piped
                                    )


Training RandomForest


In [11]:
predictions, score = predict_testset(model_instance[0], 
                                ts_test_piped[config.longest_ts_test_idx], 
                                ts_test_weather_piped[config.longest_ts_test_idx],
                                config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                )

Predicting test set...
2017-12-02 23:00:00 2017-12-03 00:00:00
2017-12-03 23:00:00 2017-12-04 00:00:00
2017-12-04 23:00:00 2017-12-05 00:00:00
2017-12-05 23:00:00 2017-12-06 00:00:00
2017-12-06 23:00:00 2017-12-07 00:00:00
2017-12-07 23:00:00 2017-12-08 00:00:00
2017-12-08 23:00:00 2017-12-09 00:00:00
2017-12-09 23:00:00 2017-12-10 00:00:00
2017-12-10 23:00:00 2017-12-11 00:00:00
2017-12-11 23:00:00 2017-12-12 00:00:00
2017-12-12 23:00:00 2017-12-13 00:00:00
2017-12-13 23:00:00 2017-12-14 00:00:00
2017-12-14 23:00:00 2017-12-15 00:00:00
2017-12-15 23:00:00 2017-12-16 00:00:00
2017-12-16 23:00:00 2017-12-17 00:00:00
2017-12-17 23:00:00 2017-12-18 00:00:00
2017-12-18 23:00:00 2017-12-19 00:00:00
2017-12-19 23:00:00 2017-12-20 00:00:00
2017-12-20 23:00:00 2017-12-21 00:00:00
2017-12-21 23:00:00 2017-12-22 00:00:00
2017-12-22 23:00:00 2017-12-23 00:00:00
2017-12-23 23:00:00 2017-12-24 00:00:00
2017-12-24 23:00:00 2017-12-25 00:00:00
2017-12-25 23:00:00 2017-12-26 00:00:00
2017-12-26 23:00:

In [12]:
px.line(predictions)